# Global Coastal Transect System

Cross-shore coastal transects are essential to coastal monitoring, offering a consistent reference line to measure coastal change, while providing a robust foundation to map coastal characteristics and derive coastal statistics thereof. The Global Coastal Transect System consists of more than 11 million cross-shore coastal transects uniformly spaced at 100-m intervals alongshore, for all OpenStreetMap coastlines that are longer than 5 kilometers. 

The dataset is extensively described in Calkoen et al., 2024, Enabling coastal analytics at planetary scale, Environmental Modelling & Software, that is available at [https://doi.org/10.1016/j.envsoft.2024.106257](https://doi.org/10.1016/j.envsoft.2024.106257); please cite this paper when the data is used. 

## By using STAC and GeoPandas 

In [72]:
import os

import dask

dask.config.set({"dataframe.query-planning": False})

import dask_geopandas
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from ipyleaflet import Map, basemaps

storage_options = {"account_name": "coclico"}

### Connect to the CoCliCo STAC 

In [73]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
gcts_collection = coclico_catalog.get_child("gcts")
gcts_collection

<Collection id=gcts>

### The dataset is geospatially partitioned

In [74]:
from coastpy.stac.utils import read_snapshot

gcts_extents = read_snapshot(
    gcts_collection,
    columns=["geometry", "assets"],
    add_href=True,  
    storage_options=storage_options,
)
gcts_extents[["geometry", "href"]].explore()

### Use a dynamic map to extract data by region of interest

The IPyleaflet map below can be used to find the bbox coordinates of a certain region.
Zoom to the area where you want to extract data and run the next cell. Please wait until the map is rendered; otherwise the coordinates cannot be extracted. 

In [75]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 15.827, -95.96
m.zoom = 15
m.layout.height = "800px"
m

Map(center=[15.827, -95.96], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [76]:
west, south, east, north = m.west, m.south, m.east, m.north
# Note: small little hack to ensure the notebook also works when running all cells at once
if not west:
    west, south, east, north = (
        30.28415679931641,
        31.276790311057272,
        30.630912780761722,
        31.51123970051334,
    )
roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

### Find the data partitions that span the region of interest

In [77]:
hrefs = gpd.sjoin(gcts_extents, roi).href.to_list()

## Read the data from cloud storage

In [78]:
transects = dask_geopandas.read_parquet(hrefs, storage_options=storage_options)
transects = (
    transects.sjoin(roi.to_crs(transects.crs)).drop(columns=["index_right"]).compute()
)

transects.head()

,transect_id,lon,lat,bearing,geometry,osm_coastline_is_closed,osm_coastline_length,utm_epsg,bbox,quadkey,continent,country,common_country_name,common_region_name
157846,cl32408s02tr00950470,7.238595,53.331287,191.597839,"LINESTRING (7.24125 53.34013, 7.23594 53.32244)",False,1073341,32632,"{'xmax': 7.241250626441005, 'xmin': 7.23594112...",020202013131,EU,DE,Germany,Lower Saxony
157847,cl32408s02tr00950570,7.240057,53.331085,195.541443,"LINESTRING (7.24372 53.3398, 7.23639 53.32237)",False,1073341,32632,"{'xmax': 7.243723409469021, 'xmin': 7.23639335...",020202013131,EU,DE,Germany,Lower Saxony
157849,cl32408s02tr00950670,7.241497,53.330833,201.142776,"LINESTRING (7.24657 53.33929, 7.23643 53.32237)",False,1073341,32632,"{'xmax': 7.246566898356305, 'xmin': 7.23642920...",020202013131,EU,DE,Germany,Lower Saxony
157868,cl32408s02tr00950770,7.242889,53.330502,209.095825,"LINESTRING (7.24987 53.33846, 7.23591 53.32254)",False,1073341,32632,"{'xmax': 7.249865783299761, 'xmin': 7.23591458...",020202013131,EU,DE,Germany,Lower Saxony
157869,cl32408s02tr00950870,7.244195,53.330059,217.107391,"LINESTRING (7.25296 53.33736, 7.23544 53.32276)",False,1073341,32632,"{'xmax': 7.2529567146462055, 'xmin': 7.2354356...",020202013131,EU,DE,Germany,Lower Saxony


In [8]:
import colorcet as cc

transects[["geometry", "bearing"]].hvplot(
    geo=True,
    tiles="ESRI",
    color="bearing",
    frame_width=650,
    frame_height=550,
    colorbar=True,
    cmap=cc.CET_C6,
    clim=(0, 360),
    title="Transect geometries with north bearing [deg]",
    clabel="North Bearing [deg]",
)

:Overlay
   .WMTS.I :WMTS   [Longitude,Latitude]
   .Path.I :Path   [Longitude,Latitude]   (bearing)

In [98]:
import osmnx as ox
import matplotlib.pyplot as plt


# Define the directory containing the case study JSON files
casestudy_dir = "/Users/juulhemmes/Documents/Studie/Msc/Thesis/nourishment_database/data/raw"

nourishments = gpd.read_file(os.path.join(casestudy_dir, "NL_polygon.json"))
dutch_boundary = ox.geocode_to_gdf("Netherlands")
transacts = transects.to_crs(epsg=4326)

transects_nl = transects[transects['country'] == 'NL']
transects_nl.head()

,transect_id,lon,lat,bearing,geometry,osm_coastline_is_closed,osm_coastline_length,utm_epsg,bbox,quadkey,continent,country,common_country_name,common_region_name
169514,cl32408s02tr00996570,6.889399,53.382042,86.318550,"LINESTRING (6.87444 53.3812, 6.90436 53.38288)",False,1073341,32632,"{'xmax': 6.904363507235584, 'xmin': 6.87443526...",020202013212,EU,NL,Netherlands,Groningen
169524,cl32408s02tr00997070,6.888881,53.386524,88.521629,"LINESTRING (6.87387 53.38603, 6.90389 53.38702)",False,1073341,32632,"{'xmax': 6.903889783865952, 'xmin': 6.87387226...",020202013212,EU,NL,Netherlands,Groningen
169526,cl32408s02tr00995070,6.894100,53.368877,80.798912,"LINESTRING (6.87935 53.36718, 6.90886 53.37057)",False,1073341,32632,"{'xmax': 6.908855515099656, 'xmin': 6.87934559...",020202013212,EU,NL,Netherlands,Groningen
169534,cl32408s02tr00996170,6.890265,53.378487,81.675507,"LINESTRING (6.87547 53.37692, 6.90507 53.38005)",False,1073341,32632,"{'xmax': 6.90506576918056, 'xmin': 6.875466098...",020202013212,EU,NL,Netherlands,Groningen
169535,cl32408s02tr00996270,6.890007,53.379372,83.131874,"LINESTRING (6.87515 53.37803, 6.90487 53.38071)",False,1073341,32632,"{'xmax': 6.904869154323176, 'xmin': 6.87514555...",020202013212,EU,NL,Netherlands,Groningen


In [99]:
# Convert 'Begin datum' to datetime format
nourishments['Begin datum'] = pd.to_datetime(nourishments['Begin datum'], errors='coerce')

# Extract the year from the datetime column
nourishments['year'] = nourishments['Begin datum'].dt.year

# Rename the column for clarity
nourishments.rename(columns={'Volume per meter': 'volume_per_m'}, inplace=True)

nourishments["year"] = pd.to_numeric(nourishments["year"], errors="coerce").astype("Int64")

# Check the first rows to confirm the changes
print(nourishments.head())

  ID       Type         Kustvak Begin datum Eind datum           Jarkusraaien  \
0  0  vooroever  Walcheren (16)  1952-04-01 1952-10-31  16003400 t/m 16003440   
1  1     strand  Walcheren (16)  1952-01-01 1952-12-31  16003260 t/m 16003340   
2  2     strand    Delfland (9)  1953-01-01 1953-12-31    9010050 t/m 9010150   
3  3     strand    Rijnland (8)  1962-01-01 1967-12-31    8005650 t/m 8005750   
4  4       duin     Goeree (12)  1966-01-01 1966-12-31  12001500 t/m 12001700   

         Volume volume_per_m  \
0     50.000 m³     125 m³/m   
1    775.000 m³     968 m³/m   
2     70.000 m³      70 m³/m   
3  1.500.000 m³    1500 m³/m   
4    150.000 m³      75 m³/m   

                                            geometry  year  
0  POLYGON ((3.55242 51.44716, 3.55467 51.44605, ...  1952  
1  POLYGON ((3.53742 51.45685, 3.53883 51.45615, ...  1952  
2  POLYGON ((4.27187 52.10824, 4.27127 52.10785, ...  1953  
3  POLYGON ((4.56856 52.45488, 4.5675 52.45272, 4...  1962  
4  POLYGON ((3.

In [100]:
print(nourishments[["geometry", "volume_per_m", "year"]].head())
print(transects_nl[["transect_id", "geometry"]].head())

# Ensure both datasets use the same CRS
transects_nl = transects_nl.to_crs(epsg=4326)
nourishments = nourishments.to_crs(epsg=4326)


                                            geometry volume_per_m  year
0  POLYGON ((3.55242 51.44716, 3.55467 51.44605, ...     125 m³/m  1952
1  POLYGON ((3.53742 51.45685, 3.53883 51.45615, ...     968 m³/m  1952
2  POLYGON ((4.27187 52.10824, 4.27127 52.10785, ...      70 m³/m  1953
3  POLYGON ((4.56856 52.45488, 4.5675 52.45272, 4...    1500 m³/m  1962
4  POLYGON ((3.86012 51.81652, 3.85948 51.81634, ...      75 m³/m  1966
                 transect_id                                         geometry
169514  cl32408s02tr00996570   LINESTRING (6.87444 53.3812, 6.90436 53.38288)
169524  cl32408s02tr00997070  LINESTRING (6.87387 53.38603, 6.90389 53.38702)
169526  cl32408s02tr00995070  LINESTRING (6.87935 53.36718, 6.90886 53.37057)
169534  cl32408s02tr00996170  LINESTRING (6.87547 53.37692, 6.90507 53.38005)
169535  cl32408s02tr00996270  LINESTRING (6.87515 53.37803, 6.90487 53.38071)


In [101]:
# Perform a spatial join to find which transects intersect with nourishment areas
transects_nourished = gpd.sjoin(transects_nl, nourishments, how="left", predicate="intersects")

# Keep only relevant columns (modify based on your dataset)
transects_nourished = transects_nourished[['transect_id', 'geometry', 'volume_per_m', 'year']]

In [102]:
transects_nourished

,transect_id,geometry,volume_per_m,year
169514,cl32408s02tr00996570,"LINESTRING (6.87444 53.3812, 6.90436 53.38288)",NaN,<NA>
169524,cl32408s02tr00997070,"LINESTRING (6.87387 53.38603, 6.90389 53.38702)",NaN,<NA>
169526,cl32408s02tr00995070,"LINESTRING (6.87935 53.36718, 6.90886 53.37057)",NaN,<NA>
169534,cl32408s02tr00996170,"LINESTRING (6.87547 53.37692, 6.90507 53.38005)",NaN,<NA>
169535,cl32408s02tr00996270,"LINESTRING (6.87515 53.37803, 6.90487 53.38071)",NaN,<NA>
...,...,...,...,...
1434391,cl32661s00tr00000668,"LINESTRING (5.13777 53.35025, 5.11842 53.33651)",NaN,<NA>
1434392,cl32661s00tr00000768,"LINESTRING (5.13799 53.35019, 5.12063 53.33553)",NaN,<NA>
1434393,cl32663s00tr00003559,"LINESTRING (5.13159 53.29573, 5.14943 53.28128)",NaN,<NA>
1434394,cl32663s00tr00003659,"LINESTRING (5.1328 53.29626, 5.15064 53.28181)",NaN,<NA>


In [103]:
# Count NaN values in specific columns
missing_volume = transects_nourished['volume_per_m'].isna().sum()
missing_year = transects_nourished['year'].isna().sum()

print(f"Missing values in 'volume_per_m': {missing_volume}")
print(f"Missing values in 'year': {missing_year}")

Missing values in 'volume_per_m': 6937
Missing values in 'year': 6937


In [97]:
# Define the output directory relative to the notebooks folder

# Define the output file path
output_file = os.path.join(casestudy_dir, f"NL_nourishment_at_transect.json")

# Save the GeoDataFrame as a GeoJSON file
transects_nourished.to_file(output_file, driver="GeoJSON")

print(f"GeoJSON saved at: {output_file}")

GeoJSON saved at: /Users/juulhemmes/Documents/Studie/Msc/Thesis/nourishment_database/data/raw/NL_nourishment_at_transect.json
